In [1]:
from qiskit_algorithms import MinimumEigensolverResult
from qiskit_algorithms.minimum_eigensolvers import NumPyMinimumEigensolver, VQE
from qiskit_nature.second_q.transformers import FreezeCoreTransformer
from qiskit_nature.second_q.formats.molecule_info import MoleculeInfo
from qiskit_nature.second_q.mappers import ParityMapper
from qiskit_nature.second_q.circuit.library import UCCSD, HartreeFock
from qiskit_nature.second_q.drivers import PySCFDriver
from qiskit.circuit.library import EfficientSU2
import numpy as np
from qiskit_algorithms.optimizers import SLSQP , SPSA , ADAM
from qiskit_aer.primitives import Estimator
#qiskit_nature.settings.use_pauli_sum_op = False 
from qiskit_aer import Aer
from pyinstrument import Profiler
import threading
from qiskit_aer.primitives import Estimator as AerEstimator
import math
from qiskit.circuit.library import EfficientSU2
from qiskit_aer import AerSimulator
import cupy as cp

In [2]:
molecule = MoleculeInfo(
        # Coordinates in Angstrom
        symbols=["N", "H" , "H", "H" , "H"],
        coords=([0.0, 0.0, 0.0], [0.6895640, 0.6895640, 0.6895640] ,[-0.6895640, -0.6895640, 0.6895640] , [-0.6895640, 0.6895640, -0.6895640],[0.6895640, -0.6895640, -0.6895640]),
        multiplicity=0,  # = 2*spin + 1
        charge=0,
    )

In [3]:
driver = PySCFDriver.from_molecule(molecule)
properties = driver.run()

problem = FreezeCoreTransformer(
    freeze_core=True, remove_orbitals=[-3, -2]
).transform(properties)

num_particles = problem.num_particles
num_spatial_orbitals = problem.num_spatial_orbitals

mapper = ParityMapper(num_particles=num_particles)
qubit_op = mapper.map(problem.second_q_ops()[0])

print(qubit_op)



SparsePauliOp(['IIIIIIIIIIIIII', 'IIIIIIIIIIIIIZ', 'IIIIIIIIIIIIZZ', 'IIIIIIIIIIIIZI', 'IIIIIIIIZXXXXZ', 'IIIIIIIIZXXXXI', 'IIIIIIIIIYXXYI', 'IIIIIIIIIYXXYZ', 'IIIIIIIZXXXXXZ', 'IIIIIIIZXXXXXI', 'IIIIIIIIYXXXYI', 'IIIIIIIIYXXXYZ', 'IIIIIIIXXXXXXZ', 'IIIIIIIXXXXXXI', 'IIIIIIIYXXXXYI', 'IIIIIIIYXXXXYZ', 'IIIIIIIIIIIZZI', 'IIIIIIIIIIIZZZ', 'IIIIIIIIZXXXZI', 'IIIIIIIIZXXXZZ', 'IIIIIIIIIYXYII', 'IIIIIIIIIYXYIZ', 'IIIIIIIZXXXXZI', 'IIIIIIIZXXXXZZ', 'IIIIIIIIYXXYII', 'IIIIIIIIYXXYIZ', 'IIIIIIIXXXXXZI', 'IIIIIIIXXXXXZZ', 'IIIIIIIYXXXYII', 'IIIIIIIYXXXYIZ', 'IIIIIIIIIIZZII', 'IIIIIIIIIIZZIZ', 'IIIIIIIIZXXZII', 'IIIIIIIIZXXZIZ', 'IIIIIIIIIYYIII', 'IIIIIIIIIYYIIZ', 'IIIIIIIZXXXZII', 'IIIIIIIZXXXZIZ', 'IIIIIIIIYXYIII', 'IIIIIIIIYXYIIZ', 'IIIIIIIXXXXZII', 'IIIIIIIXXXXZIZ', 'IIIIIIIYXXYIII', 'IIIIIIIYXXYIIZ', 'IIIIIIIIIZZIII', 'IIIIIIIIIZZIIZ', 'IIIIIIIIZZIIII', 'IIIIIIIIZZIIIZ', 'IIIIIIIZZIIIII', 'IIIIIIIZZIIIIZ', 'IIIIIIIZIIIIII', 'IIIIIIIZIIIIIZ', 'IIIIIIZIIIIIII', 'IIIIIIZIIIIIIZ', 'IIZXXXXIIIII

In [4]:
init_state = HartreeFock( num_spatial_orbitals, num_particles, mapper)
var_form = UCCSD( num_spatial_orbitals, num_particles, mapper, initial_state=init_state)
# var_form = EfficientSU2(num_qubits=qubit_op.num_qubits, entanglement='linear' , initial_state=init_state)
print(var_form)

      »
 q_0: »
      »
 q_1: »
      »
 q_2: »
      »
 q_3: »
      »
 q_4: »
      »
 q_5: »
      »
 q_6: »
      »
 q_7: »
      »
 q_8: »
      »
 q_9: »
      »
q_10: »
      »
q_11: »
      »
q_12: »
      »
q_13: »
      »
«      ┌────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────

In [5]:

optimizer = SLSQP(maxiter=30)
aer_estimator = Estimator(approximation=True)


In [7]:
def exact_result(qubit_op, problem):

    sol = NumPyMinimumEigensolver().compute_minimum_eigenvalue(qubit_op)
    result = problem.interpret(sol)
    return result

result = exact_result(qubit_op, problem)
print(" exact energy " , result.total_energies[0].real)

 exact energy  -55.86312851955861


In [6]:
def exact_result(qubit_op, problem):
    
    matrix = qubit_op.to_matrix()
    cp_matrix = cp.array(matrix)

    eigenvalues, eigenvectors = cp.linalg.eigh(cp_matrix)
    min_index = cp.argmin(eigenvalues.real)
    min_eigenvalue = eigenvalues[min_index].real
    min_eigenvector = eigenvectors[:, min_index]

    min_eigenvalue = cp.asnumpy(min_eigenvalue)
    min_eigenvector = cp.asnumpy(min_eigenvector)

    result = MinimumEigensolverResult()
    result.eigenvalue = min_eigenvalue
    result.eigenstate = min_eigenvector

    interpreted_result = problem.interpret(result)
    return interpreted_result



result = exact_result(qubit_op, problem)
print(" exact energy " , result.total_energies[0].real)

 exact energy  -55.86312851955859


In [8]:
vqe = VQE(aer_estimator, var_form, optimizer)
vqe.initial_point = [0] * var_form.num_parameters
from qiskit_nature.second_q.algorithms import GroundStateEigensolver

algorithm = GroundStateEigensolver(mapper, vqe)

electronic_structure_result = algorithm.solve(problem)
electronic_structure_result.formatting_precision = 6
print(electronic_structure_result.groundenergy)
print(electronic_structure_result)

-22.12092133634557
=== GROUND STATE ENERGY ===
 
* Electronic ground state energy (Hartree): -69.895074
  - computed part:      -22.120921
  - FreezeCoreTransformer extracted energy part: -47.774153
~ Nuclear repulsion energy (Hartree): 14.033696
> Total ground state energy (Hartree): -55.861379
 
=== MEASURED OBSERVABLES ===
 
  0:  # Particles: 9.000 S: 0.500 S^2: 0.751 M: -0.500
 
=== DIPOLE MOMENTS ===
 
~ Nuclear dipole moment (a.u.): [0.0  0.0  0.0]
 
  0: 
  * Electronic dipole moment (a.u.): [-0.00007  -0.000097  -0.000011]
    - computed part:      [-0.00007  -0.000097  -0.000011]
    - FreezeCoreTransformer extracted energy part: [0.0  0.0  0.0]
  > Dipole moment (a.u.): [0.00007  0.000097  0.000011]  Total: 0.000121
                 (debye): [0.000179  0.000247  0.000028]  Total: 0.000307
 


In [ ]:
vqe =  VQE(aer_estimator , var_form , optimizer,initial_point=[0] * var_form.num_parameters)

vqe_calc = vqe.compute_minimum_eigenvalue(qubit_op)
vqe_result = problem.interpret(vqe_calc).total_energies[0].real